# 1.Import dependencies and set up the environment

In [ ]:

import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
from torch_geometric.nn import GATConv
import random
import copy
import plotly.graph_objects as go
from kan import KAN  

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


# 2.Define feature layers and auxiliary tool functions

In [ ]:
# Feature layering and input and output definitions
FEATURE_COLUMNS = [
    'ITG', 'BTW', 'CTR', 'ETR', 'S', 'L', 'WA', 'WFR', 'WWR', 'DN',
    'WN', 'DS', 'WS', 'H', 'V', 'a', 'b', 'OH', 'IH', 'CN', 'CA'
]
TARGET_FEATURES = FEATURE_COLUMNS
INPUT_FEATURES = FEATURE_COLUMNS
#FEATURES_LAYER1 = ['ITG', 'BTW', 'CTR', 'ETR', 'S', 'H', 'V', 'a', 'b']
#FEATURES_LAYER2 = ['L', 'WA', 'WFR', 'WWR', 'DN', 'WN', 'DS', 'WS', 'OH', 'IH', 'CN', 'CA']
FEATURES_LAYER1 = ['ITG', 'BTW', 'CTR', 'ETR', 'WA', 'WFR', 'WWR', 'WS' , 'CA']
FEATURES_LAYER2 = ['S', 'H', 'V', 'a', 'b' , 'L', 'DN', 'WN', 'DS',  'OH', 'IH', 'CN']
assert sorted(FEATURES_LAYER1 + FEATURES_LAYER2) == sorted(INPUT_FEATURES), "分层特征必须覆盖全部输入特征"

# Get binary classification feature flags
def get_zero_one_flags(df, features):
    zero_one_flags = []
    for feat in features:
        vals = df[feat].dropna().unique()
        flag = (set(vals) <= {0, 1}) and (len(set(vals)) == 2)
        zero_one_flags.append(flag)
    return zero_one_flags

class GroupedStandardScaler:
    def __init__(self):
        self.group_scalers = {}

    def fit(self, X, groups):
        self.group_scalers = {}
        groups = pd.Series(groups)
        for g in groups.unique():
            scaler = StandardScaler()
            mask = (groups == g)
            scaler.fit(X.loc[mask])
            self.group_scalers[g] = scaler
        return self

    def transform(self, X, groups):
        X_out = X.copy()
        groups = pd.Series(groups)
        for g, scaler in self.group_scalers.items():
            mask = (groups == g)
            X_out.loc[mask] = scaler.transform(X.loc[mask])
        return X_out

    def inverse_transform(self, X, groups):
        X_out = X.copy()
        groups = pd.Series(groups)
        for g, scaler in self.group_scalers.items():
            mask = (groups == g)
            X_out.loc[mask] = scaler.inverse_transform(X.loc[mask])
        return X_out


# 3.Data denormalization and loading data

In [3]:
def denormalize_batch(normed_data, group_series, scaler_dict, cols):
    out = np.zeros_like(normed_data)
    y_other_scaler = scaler_dict['y_other_scaler']
    other_features = scaler_dict['other_features']
    idx_map = {col: i for i, col in enumerate(cols)}
    if other_features:
        idxs_other = [idx_map[c] for c in cols if c not in scaler_dict['group_features']]
        if idxs_other:
            out_other = y_other_scaler.inverse_transform(normed_data[:, idxs_other])
            for j, idx in enumerate(idxs_other):
                out[:, idx] = out_other[:, j]
    for gf in scaler_dict['group_features']:
        idx = idx_map[gf]
        arr = normed_data[:, idx].reshape(-1, 1)
        if gf == 'BTW':
            arr_inv = scaler_dict['group_scaler_BTW'].inverse_transform(pd.DataFrame(arr, columns=[gf]), pd.Series(group_series)).values[:, 0]
        elif gf == 'CTR':
            arr_inv = scaler_dict['group_scaler_CTR'].inverse_transform(pd.DataFrame(arr, columns=[gf]), pd.Series(group_series)).values[:, 0]
        out[:, idx] = arr_inv
    return out

def read_data(folder_path):
    data_list = []
    file_names = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
    if not file_names:
        raise FileNotFoundError(f"No .xlsx files in {folder_path}")
    for file_name in file_names:
        file_path = os.path.join(folder_path, file_name)
        xls = pd.ExcelFile(file_path)
        for sheet_name in xls.sheet_names:
            if sheet_name == 'data':
                continue
            df = pd.read_excel(file_path, sheet_name=sheet_name)
            df['source'] = f"{file_name}_{sheet_name}"
            data_list.append(df)
    if not data_list:
        raise ValueError("No valid sheets loaded in read_data")
    return pd.concat(data_list, ignore_index=True)

# 4.Process data and construct graph data objects

In [4]:
def process_data(node_data, edge_data):
    node_data['global_id'] = range(len(node_data))
    id_map = {(row['source'], row['ID']): row['global_id'] for _, row in node_data.iterrows()}
    all_input_cols = INPUT_FEATURES
    idx_layer1 = [all_input_cols.index(col) for col in FEATURES_LAYER1]
    idx_layer2 = [all_input_cols.index(col) for col in FEATURES_LAYER2]

    group_col = 'Class'
    group_series = node_data[group_col].astype(str)
    group_features = ['BTW', 'CTR']
    other_features = [col for col in INPUT_FEATURES if col not in group_features]

    input_scaler = StandardScaler()
    X_other = input_scaler.fit_transform(node_data[other_features].fillna(0))

    group_scaler_BTW = GroupedStandardScaler()
    group_scaler_BTW.fit(node_data[['BTW']].fillna(0), group_series)
    group_scaler_CTR = GroupedStandardScaler()
    group_scaler_CTR.fit(node_data[['CTR']].fillna(0), group_series)

    X_BTW = group_scaler_BTW.transform(node_data[['BTW']].fillna(0), group_series).values
    X_CTR = group_scaler_CTR.transform(node_data[['CTR']].fillna(0), group_series).values

    X = np.zeros((len(node_data), len(all_input_cols)))
    for i, col in enumerate(all_input_cols):
        if col == 'BTW':
            X[:, i] = X_BTW[:, 0]
        elif col == 'CTR':
            X[:, i] = X_CTR[:, 0]
        else:
            X[:, i] = X_other[:, other_features.index(col)]

    y_other = node_data[[col for col in TARGET_FEATURES if col not in group_features]].fillna(0)
    y_scaler = StandardScaler()
    y_scaler.fit(y_other)
    y_other_std = y_scaler.transform(y_other)

    y_BTW = group_scaler_BTW.transform(node_data[['BTW']].fillna(0), group_series).values
    y_CTR = group_scaler_CTR.transform(node_data[['CTR']].fillna(0), group_series).values

    y = np.zeros((len(node_data), len(TARGET_FEATURES)))
    for i, col in enumerate(TARGET_FEATURES):
        if col == 'BTW':
            y[:, i] = y_BTW[:, 0]
        elif col == 'CTR':
            y[:, i] = y_CTR[:, 0]
        else:
            y[:, i] = y_other_std[:, y_other.columns.get_loc(col)]

    edge_list = []
    for _, row in edge_data.iterrows():
        src_key = (row['source'], row['StartPointID'])
        dst_key = (row['source'], row['EndPointID'])
        if src_key in id_map and dst_key in id_map:
            edge_list.append([id_map[src_key], id_map[dst_key]])
    edge_index = torch.LongTensor(edge_list).t().contiguous()

    scaler_dict = {
        'input_scaler': input_scaler,
        'group_scaler_BTW': group_scaler_BTW,
        'group_scaler_CTR': group_scaler_CTR,
        'other_features': other_features,
        'group_features': group_features,
        'group_col': group_col,
        'group_series': group_series.values,
        'y_other_scaler': y_scaler
    }
    return {
        'x': torch.FloatTensor(X),
        'y': torch.FloatTensor(y),
        'edge_index': edge_index,
        'idx_layer1': idx_layer1,
        'idx_layer2': idx_layer2,
        'scaler_dict': scaler_dict,
        'group_series': group_series.values
    }

def get_planar_faces(pyg_data):
    import networkx as nx
    G_nx = to_networkx(pyg_data, to_undirected=True)
    faces = nx.cycle_basis(G_nx)
    return faces

def build_face_adjacency(num_nodes, faces):
    from collections import defaultdict
    face_adj_pairs = defaultdict(bool)
    for f in faces:
        k = len(f)
        for idx1 in range(k):
            i = f[idx1]
            for idx2 in range(idx1 + 1, k):
                j = f[idx2]
                face_adj_pairs[(i, j)] = True
                face_adj_pairs[(j, i)] = True
    rows, cols = [], []
    for (i, j), flag in face_adj_pairs.items():
        if flag and i != j:
            rows.append(i)
            cols.append(j)
    for i in range(num_nodes):
        rows.append(i)
        cols.append(i)
    return torch.LongTensor([rows, cols])

# 5.Define the main model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.data import Data

class SplitFeatureMLP_GAT(nn.Module):
    def __init__(self, input_dim1, input_dim2, n_targets, mlp_dim=32, hidden_dim=64, dropout=0.2, heads=2, concat=True):
        super().__init__()
        # Feature projection layer
        self.proj1 = nn.Linear(input_dim1, mlp_dim)
        self.proj2 = nn.Linear(input_dim2, mlp_dim)
        
        # MLP
        self.mlp1 = nn.Sequential(
            nn.Linear(mlp_dim, mlp_dim * 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(mlp_dim * 2, mlp_dim)
        )
        self.mlp2 = nn.Sequential(
            nn.Linear(mlp_dim, mlp_dim * 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(mlp_dim * 2, mlp_dim)
        )
        
        # GAT
        self.gat1 = GATConv(mlp_dim, hidden_dim, heads=heads, concat=concat)
        self.gat2 = GATConv(mlp_dim, hidden_dim, heads=heads, concat=concat)
        
        # align
        gcn_out_dim = hidden_dim * heads if concat else hidden_dim
        self.mlp_align1 = nn.Linear(mlp_dim, gcn_out_dim)
        self.mlp_align2 = nn.Linear(mlp_dim, gcn_out_dim)
        
        # fusion_gate
        self.fusion_gate = nn.Sequential(
            nn.Linear(gcn_out_dim * 4, gcn_out_dim * 4),
            nn.Sigmoid()
        )
        self.dropout = nn.Dropout(dropout)
        
        # regressor
        self.regressor = nn.Linear(gcn_out_dim, n_targets)

    def forward(self, data):
        x = data.x
        idx1 = data.idx_layer1
        idx2 = data.idx_layer2
        edge_index_face = data.edge_index_face
        
        # 
        x1 = x[:, idx1]
        x1_proj = self.proj1(x1)
        h1_mlp = self.mlp1(x1_proj)
        h1_mlp_aligned = self.mlp_align1(h1_mlp)
        h1_gat = F.elu(self.gat1(h1_mlp, edge_index_face))
        h1_gat = self.dropout(h1_gat)
        
        # 
        x2 = x[:, idx2]
        x2_proj = self.proj2(x2)
        h2_mlp = self.mlp2(x2_proj)
        h2_mlp_aligned = self.mlp_align2(h2_mlp)
        h2_gat = F.elu(self.gat2(h2_mlp, edge_index_face))
        h2_gat = self.dropout(h2_gat)
        
        # 
        combined = torch.cat([h1_mlp_aligned, h1_gat, h2_mlp_aligned, h2_gat], dim=1)
        gate = self.fusion_gate(combined)
        gate1, gate2, gate3, gate4 = torch.chunk(gate, 4, dim=1)
        fused = (gate1 * h1_mlp_aligned) + (gate2 * h1_gat) + (gate3 * h2_mlp_aligned) + (gate4 * h2_gat)
        
        # 
        out = self.regressor(fused)
        return out

# mlp
def create_mlp_model(input_dim1, input_dim2, n_targets):
    return SplitFeatureMLP_GAT(
        input_dim1=input_dim1,
        input_dim2=input_dim2,
        n_targets=n_targets,
        mlp_dim=32,      
        hidden_dim=64,    
        dropout=0.2,     
        heads=2,         
        concat=True      
    )


if __name__ == "__main__":

    input_dim1 = 18 
    input_dim2 = 18 
    n_targets = 1  
    
    model = create_mlp_model(input_dim1, input_dim2, n_targets)
    print(model)
    
    data = Data(
        x=torch.randn(100, 50),         
        idx_layer1=torch.arange(0, 18),   
        idx_layer2=torch.arange(18, 36), 
        edge_index_face=torch.randint(0, 100, (2, 200))  
    )
    
    output = model(data)
    print("输出形状:", output.shape)  # 应输出: torch.Size([100, 1])

SplitFeatureMLP_GAT(
  (proj1): Linear(in_features=18, out_features=32, bias=True)
  (proj2): Linear(in_features=18, out_features=32, bias=True)
  (mlp1): Sequential(
    (0): Linear(in_features=32, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=64, out_features=32, bias=True)
  )
  (mlp2): Sequential(
    (0): Linear(in_features=32, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=64, out_features=32, bias=True)
  )
  (gat1): GATConv(32, 64, heads=2)
  (gat2): GATConv(32, 64, heads=2)
  (mlp_align1): Linear(in_features=32, out_features=128, bias=True)
  (mlp_align2): Linear(in_features=32, out_features=128, bias=True)
  (fusion_gate): Sequential(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): Sigmoid()
  )
  (dropout): Dropout(p=0.2, inplace=False)
  (regressor): Linear(in_features=128, out_features=1, bias=True)
)
输出形状: torch.Size([100,

# 6.Training and evaluation functions

In [6]:
def calculate_accuracy_multi(y_true, y_pred, tolerance=0.1, zero_one_flags=None):
    y_true = torch.as_tensor(y_true)
    y_pred = torch.as_tensor(y_pred)
    n_feat = y_true.shape[1]
    accs = []
    for j in range(n_feat):
        yt = y_true[:, j]
        yp = y_pred[:, j]
        if zero_one_flags and zero_one_flags[j]:
            acc = (yt.round() == yp.round()).float().mean().item()
        else:
            mask = (yt != 0)
            if mask.sum() == 0:
                acc = 0.
            else:
                acc = ((torch.abs(yt - yp) / torch.abs(yt))[mask] <= tolerance).float().mean().item()
        accs.append(acc)
    return accs

def train_and_evaluate(model, data, zero_one_flags, epochs=200):
    criterion = torch.nn.SmoothL1Loss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)
    history = {
        'train_loss': [], 'val_loss': [], 'test_loss': [],
        'train_mae': [], 'val_mae': [], 'test_mae': [],
        'acc10_train': [], 'acc10_val': [], 'acc10_test': [],
        'acc5_train': [], 'acc5_val': [], 'acc5_test': []
    }
    best_val_loss = float('inf')
    best_epoch = -1
    best_state = None
    best_pred_train = best_true_train = None

    scaler_dict = data.scaler_dict
    group_series = data.group_series
    cols = TARGET_FEATURES

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        pred = model(data)
        train_loss = criterion(pred[data.train_mask], data.y[data.train_mask])
        train_loss.backward()
        optimizer.step()
        scheduler.step()

        model.eval()
        with torch.no_grad():
            pred = model(data)
            train_loss_eval = criterion(pred[data.train_mask], data.y[data.train_mask]).item()
            val_loss = criterion(pred[data.val_mask], data.y[data.val_mask]).item()
            test_loss = criterion(pred[data.test_mask], data.y[data.test_mask]).item()
            pred_denorm = torch.tensor(denormalize_batch(pred.detach().cpu().numpy(), group_series, scaler_dict, cols))
            true_denorm = torch.tensor(denormalize_batch(data.y.detach().cpu().numpy(), group_series, scaler_dict, cols))
            train_mae = torch.abs(pred_denorm[data.train_mask] - true_denorm[data.train_mask]).mean().item()
            val_mae = torch.abs(pred_denorm[data.val_mask] - true_denorm[data.val_mask]).mean().item()
            test_mae = torch.abs(pred_denorm[data.test_mask] - true_denorm[data.test_mask]).mean().item()
            train_acc10 = calculate_accuracy_multi(true_denorm[data.train_mask], pred_denorm[data.train_mask], 0.1, zero_one_flags)
            train_acc5 = calculate_accuracy_multi(true_denorm[data.train_mask], pred_denorm[data.train_mask], 0.05, zero_one_flags)
            val_acc10 = calculate_accuracy_multi(true_denorm[data.val_mask], pred_denorm[data.val_mask], 0.1, zero_one_flags)
            val_acc5 = calculate_accuracy_multi(true_denorm[data.val_mask], pred_denorm[data.val_mask], 0.05, zero_one_flags)
            test_acc10 = calculate_accuracy_multi(true_denorm[data.test_mask], pred_denorm[data.test_mask], 0.1, zero_one_flags)
            test_acc5 = calculate_accuracy_multi(true_denorm[data.test_mask], pred_denorm[data.test_mask], 0.05, zero_one_flags)

        history['train_loss'].append(train_loss_eval)
        history['val_loss'].append(val_loss)
        history['test_loss'].append(test_loss)
        history['train_mae'].append(train_mae)
        history['val_mae'].append(val_mae)
        history['test_mae'].append(test_mae)
        history['acc10_train'].append(train_acc10)
        history['acc10_val'].append(val_acc10)
        history['acc10_test'].append(test_acc10)
        history['acc5_train'].append(train_acc5)
        history['acc5_val'].append(val_acc5)
        history['acc5_test'].append(test_acc5)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_epoch = epoch
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            best_pred_train = pred_denorm[data.train_mask].detach().cpu().numpy()
            best_true_train = true_denorm[data.train_mask].detach().cpu().numpy()

        def acc_str(lst):
            return '[' + ', '.join(f"{v:.2f}" for v in lst) + ']'

        # 增加一行打印特征名称，每行与后续准确率对齐
        feat_str = '[' + ', '.join(f"{name:>4}" for name in TARGET_FEATURES) + ']'

        print(f"Epoch {epoch + 1}/{epochs} | "
            f"Train Loss: {train_loss_eval:.4f} | Val Loss: {val_loss:.4f} | Test Loss: {test_loss:.4f} | "
            f"Train MAE: {train_mae:.4f} | Val MAE: {val_mae:.4f} | Test MAE: {test_mae:.4f}")

        print(f"   Features:    {feat_str}")
        print(f"   ACC10-train: {acc_str(train_acc10)} | mean={np.mean(train_acc10):.3f}")
        print(f"   ACC10-val:   {acc_str(val_acc10)} | mean={np.mean(val_acc10):.3f}")
        print(f"   ACC10-test:  {acc_str(test_acc10)} | mean={np.mean(test_acc10):.3f}")
        print(f"   ACC5-train:  {acc_str(train_acc5)} | mean={np.mean(train_acc5):.3f}")
        print(f"   ACC5-val:    {acc_str(val_acc5)} | mean={np.mean(val_acc5):.3f}")
        print(f"   ACC5-test:   {acc_str(test_acc5)} | mean={np.mean(test_acc5):.3f}")

    if best_state is not None:
        model.load_state_dict(best_state)
    return history, best_pred_train, best_true_train, best_epoch


# 7.Prediction Plotting and Feature Importance

In [ ]:
def plot_predictions(true_train, pred_train, colnames, output_root='outputs006'):
    os.makedirs(output_root, exist_ok=True)
    for colidx, colname in enumerate(colnames):
        subdir = os.path.join(output_root, colname)
        os.makedirs(subdir, exist_ok=True)
        sorted_idx = np.argsort(true_train[:, colidx])
        sorted_true = true_train[sorted_idx, colidx]
        sorted_pred = pred_train[sorted_idx, colidx]

        display_samples = len(sorted_true)

        sliced_true = sorted_true[:display_samples]
        sliced_pred = sorted_pred[:display_samples]

        plt.figure(figsize=(12, 6))
        plt.plot(sliced_true, label='True Values', color='green', alpha=0.8)
        plt.plot(sliced_pred, label='Predictions', color='red', alpha=0.6)
        plt.fill_between(range(len(sliced_true)), sliced_true * 0.9, sliced_true * 1.1,
                         color='blue', alpha=0.1, label='±10% Range')
        plt.title(f"Prediction {colname} (All Samples, ±10%)")
        plt.xlabel("Sorted Samples")
        plt.ylabel(colname)
        plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(subdir, f"{colname}_±10%.png"), dpi=600)
        plt.close()

        plt.figure(figsize=(12, 6))
        plt.plot(sliced_true, label='True Values', color='green', alpha=0.8)
        plt.plot(sliced_pred, label='Predictions', color='red', alpha=0.6)
        plt.fill_between(range(len(sliced_true)), sliced_true * 0.95, sliced_true * 1.05,
                         color='blue', alpha=0.1, label='±5% Range')
        plt.title(f"Prediction {colname} (All Samples, ±5%)")
        plt.xlabel("Sorted Samples")
        plt.ylabel(colname)
        plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(subdir, f"{colname}_±5%.png"), dpi=600)
        plt.close()

        # ======= plotly draw 10% =======
        fig10 = go.Figure()

        fig10.add_trace(go.Scatter(
            x=list(range(len(sliced_true))),
            y=sliced_true * 0.90,
            mode='lines',
            name='-10%',
            line=dict(width=0, color='rgba(90,170,255,0)'),  
            showlegend=True
        ))

        fig10.add_trace(go.Scatter(
            x=list(range(len(sliced_true))),
            y=sliced_true * 1.10,
            mode='lines',
            name='+10% Range',
            fill='tonexty',
            fillcolor='rgba(90,170,255,0.3)',    
            line=dict(width=0, color='rgba(90,170,255,0)'), 
            hoverinfo='skip',  
            showlegend=True
        ))

        fig10.add_trace(go.Scatter(
            x=list(range(len(sliced_true))),
            y=sliced_true,
            mode='lines',
            name='True Values',
            line=dict(color='green', width=1.5),
            showlegend=True
        ))
  
        fig10.add_trace(go.Scatter(
            x=list(range(len(sliced_pred))),
            y=sliced_pred,
            mode='lines',
            name='Predictions',
            line=dict(color='red', width=0.75),
            showlegend=True
        ))

        fig10.update_layout(
            title=f"Prediction {colname} (All Samples, ±10%)",
            xaxis_title="Sorted Samples",
            yaxis_title=colname,
            legend=dict(x=0.01, y=0.99),
            width=1800, height=750,
            autosize=False,

        )

        fig10.write_html(os.path.join(subdir, f"{colname}_±10%_interactive.html"))

        # ----  Plotly draw: ±5% ----
        fig5 = go.Figure()

        fig5.add_trace(go.Scatter(
            x=list(range(len(sliced_true))),
            y=sliced_true * 0.95,
            mode='lines',
            name='-5%',
            line=dict(width=0, color='rgba(90,170,255,0)'),
            showlegend=True
        ))

        fig5.add_trace(go.Scatter(
            x=list(range(len(sliced_true))),
            y=sliced_true * 1.05,
            mode='lines',
            name='+5% Range',
            fill='tonexty',
            fillcolor='rgba(90,170,255,0.3)',
            line=dict(width=0, color='rgba(90,170,255,0)'),
            hoverinfo='skip',
            showlegend=True
        ))
 
        fig5.add_trace(go.Scatter(
            x=list(range(len(sliced_true))),
            y=sliced_true,
            mode='lines',
            name='True Values',
            line=dict(color='green', width=1.5),
            showlegend=True
        ))

        fig5.add_trace(go.Scatter(
            x=list(range(len(sliced_pred))),
            y=sliced_pred,
            mode='lines',
            name='Predictions',
            line=dict(color='red', width=0.75),
            showlegend=True
        ))

        fig5.update_layout(
            title=f"Prediction {colname} (All Samples, ±5%)",
            xaxis_title="Sorted Samples",
            yaxis_title=colname,
            legend=dict(x=0.01, y=0.99),
            width=1800, height=750,
            autosize=False,

        )

        fig5.write_html(os.path.join(subdir, f"{colname}_±5%_interactive.html"))

def permutation_feature_importance(model, pyg_data, zero_one_flags, cols=None, mask=None, metric='mae'):
    model.eval()
    if cols is None:
        cols = TARGET_FEATURES
    if mask is None:
        mask = torch.ones(pyg_data.x.size(0), dtype=torch.bool)

    with torch.no_grad():
        y_pred = model(pyg_data)
    pred_denorm = torch.tensor(denormalize_batch(y_pred.cpu().numpy(), pyg_data.group_series, pyg_data.scaler_dict, cols))
    true_denorm = torch.tensor(denormalize_batch(pyg_data.y.cpu().numpy(), pyg_data.group_series, pyg_data.scaler_dict, cols))
    base_mae = torch.abs(pred_denorm[mask] - true_denorm[mask]).mean(dim=0).numpy()
    base_mae_mean = base_mae.mean()

    results = []
    importance_vals = []
    importances_dict = {}
    print("-------- Feature Permutation Importance --------")
    for idx, feat in enumerate(cols):
        X_permute = pyg_data.x.detach().clone()
        idxs = torch.randperm(X_permute.shape[0])
        X_permute[:, idx] = X_permute[:, idx][idxs]
        data_new = copy.copy(pyg_data)
        data_new.x = X_permute
        with torch.no_grad():
            y_pred_perm = model(data_new)
        pred_denorm_perm = torch.tensor(denormalize_batch(y_pred_perm.cpu().numpy(), pyg_data.group_series, pyg_data.scaler_dict, cols))
        perm_mae = torch.abs(pred_denorm_perm[mask] - true_denorm[mask]).mean(dim=0).numpy()
        delta = (perm_mae - base_mae)
        delta_mean = delta.mean()
        results.append((feat, delta_mean))
        print(f"{feat:10s} importance (MAE +): {delta_mean:.4f}")
        importance_vals.append(delta_mean)
        importances_dict[feat] = delta_mean

    feature_importance = pd.DataFrame({'feature': cols, 'importance': importance_vals})
    feature_importance = feature_importance.sort_values('importance', ascending=False).reset_index(drop=True)
    print('--------- Sorted Feature Importance ---------')
    print(feature_importance)
    return results, feature_importance

# 8.Save node information

In [ ]:
def save_node_sheet_features_full(
    node_df, y_true, y_pred, feature_names, scaler_dict, group_col='source',
    outdir='output_result/node_pred'
):

    import os
    os.makedirs(outdir, exist_ok=True)

    y_pred_real = denormalize_batch(
        normed_data=y_pred, group_series=scaler_dict['group_series'],
        scaler_dict=scaler_dict, cols=feature_names
    )
    y_true_real = denormalize_batch(
        normed_data=y_true, group_series=scaler_dict['group_series'],
        scaler_dict=scaler_dict, cols=feature_names
    )

    df = node_df.reset_index(drop=True).copy()

    for i, feat in enumerate(feature_names):
        pred_col = f"pred_{feat}"
        err_col = f"errpct_{feat}"
        df[pred_col] = y_pred_real[:, i]
        err_pct = []
        for true_val, pred_val in zip(y_true_real[:, i], y_pred_real[:, i]):
            if true_val == 0:
                err_pct.append('')
            else:
                err_pct.append(abs((pred_val - true_val) / true_val) * 100)
        df[err_col] = err_pct


    for sheet, sub_df in df.groupby(group_col):
        sub_df = sub_df.sort_values("global_id") if "global_id" in sub_df else sub_df
        base_cols = [ "Name" , "global_id", "id", "depth"] 
        included_cols = [c for c in base_cols if c in sub_df.columns]
        out_cols = included_cols + feature_names + [f"pred_{f}" for f in feature_names] + [f"errpct_{f}" for f in feature_names]
        out_path = os.path.join(outdir, f"{sheet}.csv")
        sub_df[out_cols].reset_index(drop=True).to_csv(out_path, index=False)
        print(f"[OK] 节点预测数据已保存: {out_path}")


# 9.Main program

In [ ]:
import os
import numpy as np
import torch
from torch_geometric.data import Data
import matplotlib.pyplot as plt

if __name__ == "__main__":
    # Step 1:
    node_df = read_data('nodedata')
    edge_df = read_data('edgedata')
    zero_one_flags = get_zero_one_flags(node_df, TARGET_FEATURES)
    processed_data = process_data(node_df, edge_df)

    pyg_data = Data(
        x=processed_data['x'],
        edge_index=processed_data['edge_index'],
        y=processed_data['y'],
        scaler_dict=processed_data['scaler_dict'],
        group_series=processed_data['group_series']
    )
    pyg_data.idx_layer1 = torch.tensor(processed_data['idx_layer1'])
    pyg_data.idx_layer2 = torch.tensor(processed_data['idx_layer2'])

    print(f"y shape: {pyg_data.y.shape}, mean={pyg_data.y.mean():.4f}, std={pyg_data.y.std():.4f}, min={pyg_data.y.min():.4f}, max={pyg_data.y.max():.4f}")

    # Face adjacency 
    faces = get_planar_faces(pyg_data)
    edge_index_face = build_face_adjacency(pyg_data.num_nodes, faces)
    pyg_data.edge_index_face = edge_index_face

    # Generate training/validation/test masks
    num_nodes = pyg_data.num_nodes
    indices = np.arange(num_nodes)
    np.random.seed(42)
    np.random.shuffle(indices)
    n_train = int(0.6 * num_nodes)
    n_val = int(0.2 * num_nodes)
    train_indices = indices[:n_train]
    val_indices = indices[n_train:n_train + n_val]
    test_indices = indices[n_train + n_val:]

    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)
    train_mask[train_indices] = True
    val_mask[val_indices] = True
    test_mask[test_indices] = True
    pyg_data.train_mask = train_mask
    pyg_data.val_mask = val_mask
    pyg_data.test_mask = test_mask


    model = SplitFeatureMLP_GAT(
        input_dim1=len(processed_data['idx_layer1']),      
        input_dim2=len(processed_data['idx_layer2']),      
        n_targets=pyg_data.y.shape[-1],                   
        mlp_dim=32,
        hidden_dim=64,
        dropout=0.1
    )

    # =========== Train ============
    history, pred_train, true_train, best_epoch = train_and_evaluate(model, pyg_data, zero_one_flags)
    print(f"Best epoch: {best_epoch+1}")

    # =======================
    plot_predictions(true_train, pred_train, TARGET_FEATURES, output_root='outputs006(dual+gat+mpl_plus)')

    # =========== Draw ==============
    os.makedirs('outputs006', exist_ok=True)
    plt.figure(figsize=(12, 6))
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Val Loss')
    plt.plot(history['test_loss'], label='Test Loss')
    plt.title("Loss Metrics")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.savefig("outputs006/loss_metrics.png", dpi=600, bbox_inches='tight')
    plt.close()
    plt.figure(figsize=(12, 6))
    plt.plot(history['train_mae'], label='Train MAE')
    plt.plot(history['val_mae'], label='Val MAE')
    plt.plot(history['test_mae'], label='Test MAE')
    plt.title("MAE Metrics")
    plt.xlabel("Epoch")
    plt.ylabel("MAE")
    plt.legend()
    plt.savefig("outputs006/mae_metrics.png", dpi=600, bbox_inches='tight')
    plt.close()

    # =========== Permutation =============
    print("\n=== 特征Permutation Importance分析（全部样本）===")
    results, feature_importance = permutation_feature_importance(
        model,
        pyg_data,
        zero_one_flags,
        cols=TARGET_FEATURES,
        mask=torch.ones(pyg_data.x.size(0), dtype=torch.bool)
    )
    feature_importance.to_csv("outputs006/feature_permutation_importance.csv", index=False)
    plt.figure(figsize=(12, 6))
    plt.barh(feature_importance['feature'], feature_importance['importance'])
    plt.title("Permutation Feature Importance (higher = more important)")
    plt.xlabel("Importance (MAE delta)")
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.savefig("outputs006/feat_importance_bar.png", dpi=600)
    plt.close()

    # =========== Model inference + saving inference results ============
    model.eval()
    with torch.no_grad():
        y_pred = model(pyg_data).cpu().numpy()
    y_true = pyg_data.y.cpu().numpy() if hasattr(pyg_data.y, "cpu") else np.array(pyg_data.y)

    save_node_sheet_features_full(
        node_df=node_df,
        y_true=y_true,
        y_pred=y_pred,
        feature_names=TARGET_FEATURES,
        scaler_dict=processed_data['scaler_dict'],
        group_col='source',
        outdir='outputs006/node_pred'
    )


y shape: torch.Size([2722, 21]), mean=-0.0000, std=1.0000, min=-3.8645, max=48.1045
Epoch 1/200 | Train Loss: 0.2872 | Val Loss: 0.2823 | Test Loss: 0.3134 | Train MAE: 21.0450 | Val MAE: 20.5195 | Test MAE: 22.8164
   Features:    [ ITG,  BTW,  CTR,  ETR,    S,    L,   WA,  WFR,  WWR,   DN,   WN,   DS,   WS,    H,    V,    a,    b,   OH,   IH,   CN,   CA]
   ACC10-train: [0.33, 0.24, 0.12, 0.62, 0.10, 0.19, 0.21, 0.12, 0.06, 0.27, 0.08, 0.12, 0.08, 0.80, 0.09, 0.08, 0.16, 0.67, 0.01, 0.17, 0.09] | mean=0.219
   ACC10-val:   [0.35, 0.25, 0.09, 0.62, 0.09, 0.17, 0.21, 0.12, 0.09, 0.26, 0.09, 0.08, 0.09, 0.79, 0.10, 0.10, 0.17, 0.68, 0.00, 0.15, 0.08] | mean=0.218
   ACC10-test:  [0.36, 0.24, 0.11, 0.62, 0.09, 0.20, 0.22, 0.12, 0.06, 0.25, 0.07, 0.10, 0.07, 0.83, 0.09, 0.09, 0.16, 0.69, 0.01, 0.19, 0.11] | mean=0.223
   ACC5-train:  [0.16, 0.13, 0.07, 0.34, 0.04, 0.09, 0.09, 0.06, 0.04, 0.03, 0.07, 0.03, 0.04, 0.80, 0.05, 0.05, 0.12, 0.67, 0.00, 0.05, 0.05] | mean=0.142
   ACC5-val:    [

# 10.Model inference and node prediction export

In [ ]:

model.eval()
with torch.no_grad():
    y_pred = model(pyg_data).cpu().numpy()
if hasattr(pyg_data.y, "cpu"):
    y_true = pyg_data.y.cpu().numpy()
else:
    y_true = np.array(pyg_data.y)

save_node_sheet_features_full(
    node_df=node_df,
    y_true=y_true,
    y_pred=y_pred,
    feature_names=TARGET_FEATURES,
    scaler_dict=processed_data['scaler_dict'],
    group_col='source',
    outdir='output_result/node_pred'
)



[OK] 节点预测数据已保存: output_result/node_pred\化工学院.xlsx_1f.csv
[OK] 节点预测数据已保存: output_result/node_pred\化工学院.xlsx_2f.csv
[OK] 节点预测数据已保存: output_result/node_pred\化工学院.xlsx_3f.csv
[OK] 节点预测数据已保存: output_result/node_pred\化工学院.xlsx_4f.csv
[OK] 节点预测数据已保存: output_result/node_pred\化工学院.xlsx_5f.csv
[OK] 节点预测数据已保存: output_result/node_pred\土木学院.xlsx_1f.csv
[OK] 节点预测数据已保存: output_result/node_pred\土木学院.xlsx_2f.csv
[OK] 节点预测数据已保存: output_result/node_pred\土木学院.xlsx_3f.csv
[OK] 节点预测数据已保存: output_result/node_pred\土木学院.xlsx_4f.csv
[OK] 节点预测数据已保存: output_result/node_pred\土木学院.xlsx_5f.csv
[OK] 节点预测数据已保存: output_result/node_pred\建筑与设计学院.xlsx_-1f.csv
[OK] 节点预测数据已保存: output_result/node_pred\建筑与设计学院.xlsx_1f.csv
[OK] 节点预测数据已保存: output_result/node_pred\建筑与设计学院.xlsx_2f.csv
[OK] 节点预测数据已保存: output_result/node_pred\建筑与设计学院.xlsx_3f.csv
[OK] 节点预测数据已保存: output_result/node_pred\建筑与设计学院.xlsx_4f.csv
[OK] 节点预测数据已保存: output_result/node_pred\建筑与设计学院.xlsx_5f.csv
[OK] 节点预测数据已保存: output_result/node_pred\建筑与设计学院.xlsx_6f.csv
[OK] 节点预测